In [1]:
%load_ext sql
import socket
import json

# Database migration with the Liberatii Data Platform

The Liberatii Data Platform virtualises a PostgreSQL database to enable Oracle applications to be migrated to PostgreSQL without modification. This notebook contains a tutorial and demonstration of the deployment and use of Liberatii Data Platform to migrate and test an Oracle database.

This is composed of the following steps:

1. **Deployment**<br/>
   The deployment of an Azure Managed Application to provide the Liberatii Data Platform
3. **Migration**<br/>
   The migration of the schema and data from the Oracle database to PostgreSQL
5. **Synchronisation**<br/>
   Setup of a Change Data Capture pipeline to synchronise the Oracle and PostgreSQL databases
6. **Replay testing**<br/>
   Testing of an Oracle Workload Replay against both the Oracle and PostgreSQL databases to verify proper operation

The following cell contains the connection information for the Oracle (Source) and PostgreSQL (Target) databases. It is initially setup to migrate the Oracle demo HR schema and expects the given users to exist with the required permissions on the specified databases.

In [2]:
## Hostnames of the databases
PG_HOST='postgres'
ORACLE='oracle'

## Connection data
DB='pdborcl'
USER='HR'
PSWD='hr'
ORACLE_PORT=1521
PG_PORT=5432
LGW_PORT=15432

## Connection strings (derived from the above connection data)
ORACLE_CONN_STR=f'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={ORACLE})(PORT={ORACLE_PORT}))(CONNECT_DATA=(SERVICE_NAME={DB})))'
PG=f'postgresql://{USER}:{PSWD}@{PG_HOST}:{PG_PORT}/{DB}'
ORACLE=f'oracle://{USER}:{PSWD}@{ORACLE_CONN_STR}'

print(f"""
Connection settings:

    PostgreSQL:             
      {PG}
    Oracle:                 
      {ORACLE}
""")


Connection settings:

    PostgreSQL:             
      postgresql://HR:hr@postgres:5432/pdborcl
    Oracle:                 
      oracle://HR:hr@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=oracle)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=pdborcl)))



This Notebook uses the SQL extension so connections the databases can be queried directly. The following cells check the connections to Oracle and PostgreSQL are working:

In [3]:
%%sql {ORACLE}
-- Find version information for the Oracle database
select banner_full FROM v$version

0 rows affected.


banner_full
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - ProductionVersion 19.3.0.0.0


In [4]:
%%sql {PG}
-- Find version information for the PostgreSQL database
select version()

1 rows affected.


version
"PostgreSQL 14.9 (Debian 14.9-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit"


# Stage 1: Deploy Liberatii Data Platform and Gateway

The Liberatii Data Platform and Gateway are available as a single Azure Managed Application. In this notebook we will use the Azure command line to deploy the managed application.

## Login

The following cell ensures that the user is logged into Azure using the `az` command line tool.

In [5]:
import json
account=!az account list
if account[0].find("ERROR") != -1:
    _ = !az login
    account=!az account list
if account[0].find("ERROR") != -1:
    print("Not logged in: " + "\n".join(account))
else:
    print("Logged in, available subscriptions:")
    print("\n".join([
        f"  {e['id']}: {e['name']}"
        for e in json.loads("".join(account))
        if e['state'] == 'Enabled'
    ]))

Logged in, available subscriptions:
  d2e666be-5fde-451b-84c9-9c545b3e435c: Microsoft Azure Sponsorship main
  ace5ef55-d452-4062-bdec-290eece9c71e: Pay-As-You-Go Dev/Test
  de2038ee-22e2-426a-bac2-5f277e20c0cd: MAS Demo


## Parameters

The next cell defines the parameters for creation of the Liberatii managed application. We will define a new resource group for this.

In [6]:
RESOURCE_GROUP='NotebookTest'

definitionid = !az managedapp definition show \
    --subscription "Pay-As-You-Go Dev/Test" \
    -g vmonlypgtransmanapp0v123 -n vmonlypgTranslator_0v123 \
    --query id --output tsv
subscriptionid = !az account show --query id --output tsv
resourceid = f"/subscriptions/{subscriptionid[0]}/resourceGroups/{RESOURCE_GROUP}-mrg"
params = json.dumps([{
    "virtualMachinePrefix": {"value": "mftde"},
    "virtualNetworkRange": {"value": "192.168.1.0/24"},
    "virtualMachineSize": {"value": "Standard_F2s_v2"},
    "poolMode": {"value": "session"},
    "port": {"value": LGW_PORT},
    "postgresqlHost": {"value": PG_HOST},
    "postgresqlPassword":{"value": USER},
    "postgresqlUsername":{"value": PSWD},
    "interconnectPassword":{"value":"someSecureString"},
    "infoQueryPassword":{"value":"someSecureString"},
    "customerParameters":{"value":""}
}])

## Creation

The resource group and managed application can now be created.

In [7]:
groups=!az group list --query "[?name=='{RESOURCE_GROUP}'].id"
if len(json.loads("".join(groups))) > 0:
    print("Group already exists, skipping deployment")
else:
    !az group create -g {RESOURCE_GROUP} -l ukwest
    !az managedapp create -n TestTranslator -g {RESOURCE_GROUP} \
        --location ukwest \
        --managedapp-definition-id "{definitionid[0]}" \
        --kind ServiceCatalog \
        --managed-rg-id "{resourceid}" \
        --parameters '{params}'

Group already exists, skipping deployment


## Connections

The following cell will construct connection strings for the newly deployed Liberatii Data Platform and Gateway.

If the application was not deployed other values may be used to access the Liberatii Gateway and Data Platform using the `LGW_HOST` and `PLATFORM` variables directly.

In [9]:
## Use these values if the deployment stage was skipped
LGW_HOST='pgtranslator'
PLATFORM='migration'

## ...otherwise, if there is a deployment, collect the address
ips=!az network public-ip list -g {RESOURCE_GROUP}-mrg --query [].ipAddress
if ips[0][0] == '[':
    LGW_HOST=json.loads("".join(ips))[0]
    PLATFORM=LGW_HOST
else:
    print(f"Ignoring resource group: {ips}")
    
LGW=f'postgresql://{USER}:{PSWD}@{LGW_HOST}:{LGW_PORT}/{DB}'
API_PREFIX=f"http://{socket.gethostbyname(PLATFORM)}:3000"

print(f"""
Connection settings:

    Liberatii Gateway:
      {LGW}
    Liberatii Data Platform:
      {API_PREFIX}/api
    
The API may be opened in a browser for reference.
""")

Ignoring resource group: ["ERROR: (ResourceGroupNotFound) Resource group 'NotebookTest-mrg' could not be found.", 'Code: ResourceGroupNotFound', "Message: Resource group 'NotebookTest-mrg' could not be found."]

Connection settings:

    Liberatii Gateway:
      postgresql://HR:hr@pgtranslator:15432/pdborcl
    Liberatii Data Platform:
      http://192.168.32.6:3000/api
    
The API may be opened in a browser for reference.



## Test

With the Liberatii Gateway deployed (or otherwise available) we can now test the connection.

The following command uses PL/SQL syntax with will be translated by the virtualisation layer to access the PostgreSQL database.

In [10]:
%%sql {LGW}
select BANNER FROM v$version

1 rows affected.


BANNER
"PostgreSQL 14.9 (Debian 14.9-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit"


# Stage 2. Migration

The following section shows the migration of the schema and data from the Oracle database to PostgreSQL. This uses the Liberatii Data Platform to perform the following steps:

1. Configuration, connection setup and initialisation
3. Schema Migration
4. Data Migration
5. Testing and verification

## Configuration

The following cells will construct the required connection information. Each connection (Oracle, PostgreSQL and the Liberatii Gateway) will be created in turn.

### Oracle

In [20]:
!curl {API_PREFIX}/connection -H "Content-Type: application/json" \
   -d '{{"type":"Oracle","connectionString":"{ORACLE_CONN_STR}","user":"{USER}","password":"{PSWD}","id":1}}'

{"type":"Oracle","connectionString":"(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=oracle)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=pdborcl)))","user":"HR","password":"hr","id":1}

### PostgreSQL

In [41]:
!curl {API_PREFIX}/connection -H 'Content-Type: application/json' \
  -d '{{"type":"PostgreSQL","host":"{PG_HOST}","port":5432,"database":"{DB}","user":"{USER}","password":"{PSWD}","id":2}}'

{"type":"PostgreSQL","host":"postgres","port":5432,"database":"pdborcl","user":"HR","password":"hr","id":2}

### Liberatii Gateway:

In [47]:
!curl {API_PREFIX}/connection -H 'Content-Type: application/json' \
  -d '{{"type":"LGW","host":"{LGW_HOST}","port":{LGW_PORT},"database":"{DB}","user":"{USER}","password":"{PSWD}", "id":3}}'

{"type":"LGW","host":"pgtranslator","port":15432,"database":"pdborcl","user":"HR","password":"hr","id":3}

### Configuration Parameters

The following configuration parameters are used for this demonstration. Please check the reference document to get all the available parameters.

* `dataOnePass`<br/>
  Don't use staging files or tables, just transfer all the data from the source to the target
* `user`<br/>
  List of schemas to transfer between the sources and target target
* `verbose`<br/>
  Log verbosity
* `eraseOnInit`<br/>
  Delete everything from the schema in the init stage, this is useful if we often restart the migration.

In [48]:
!curl {API_PREFIX}/config -H 'Content-Type: application/json' \
  -d '{{"dataOnePass": true, "users":["{USER}"], "verbose":4, "eraseOnInit":true}}'

{"message":"Config has been set successfully","config":{"dataOnePass":true,"verbose":4,"useCopy":true,"dataIterations":-1,"useWrapper":true,"useNative":false,"useUnlogged":false,"stat":true,"statDB":false,"rowsBuf":1000,"lightCheck":false,"dataChunkSize":-1,"bigTablesFirst":true,"debReverseOrder":false,"cli":true,"rmStagingFiles":true,"parTables":true,"hashType":"murmur","simulateUnsupportedTypes":true,"blobStreams":true,"clobStreams":false,"blobImmed":false,"clobImmed":false,"dumpCopy":false,"dryRun":false,"noBlobs":false,"idCol":"rowid","idColByTable":{},"activeSqlFiles":[],"numTries":10,"maxWait":60000,"ignoreTrim":true,"removeLastFetchFirst":false,"checkMetaData":true,"users":["HR"],"linkedServers":[],"stages":{},"workloadsFiles":"","eraseOnInit":true}}

### Initialisation

This is an initialisation operation. The framework runs simple assessments and stores the results in the target database. The operation is asynchronous: it just schedules the operation and exists immediately.

In [49]:
!curl {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "init" }}'

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Using this operation we can get the current state of the currently running operation:

In [50]:
!curl {API_PREFIX}/operation?pager=1 -H 'Content-Type: application/json'

{"title":"init","status":"Running","messages":[{"level":"Info","message":[["Type","I","Total"],["TABLE","7","7"],["PROCEDURE","2","2"],["SEQUENCE","3","3"],["INDEX","11","11"],["VIEW","1","1"],["^+Total:^:","24","^+24^"]]}],"progress":0}

And the next operatoin is the same as the previous one, except it waits until the current operation ends.

In [51]:
!curl -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"init","status":"Running","messages":[{"level":"Info","message":"\n^g^+DONE! 👍^:\n"},{"level":"Info","message":[["Type","I","Total"],["TABLE","7","7"],["PROCEDURE","2","2"],["SEQUENCE","3","3"],["INDEX","11","11"],["VIEW","1","1"],["^+Total:^:","24","^+24^"]]}],"progress":0}

## Schema migration

Now, after `init` stage is done, we have all the necessary information for migrating the database schema.

This information is stored in the PostgreSQL database allowing us to run various queries on it. For example, we can query the number of objects of each type by running with the query:


In [52]:
%%sql {PG}
select count(*), type, stage, error from dbt.migration_objects group by type, stage, error

5 rows affected.


count,type,stage,error
11,INDEX,I,None
2,PROCEDURE,I,None
7,TABLE,I,None
3,SEQUENCE,I,None
1,VIEW,I,None


If there were any errors during the execution of any stage, they will be displayed in `error` stage. The errors can be manually fixed by modifying `ddl1`, `ddl2` columns of `dbt.migration_objects` or by adding some runtime objects. 

When errors are fixed we don't need to run the whole migration from the beginning. It's enough to reset only the problematic objects' `stage` field and restart the stage. Only those objects migration will be re-run.

Now starting the schema migration, with the same parameters as with `init` stage, but with `schema` as the stage name.

The schema migration doesn't migrate constraints, indexes and triggers. They are migrated in `constraints` stage. This is done to make data migration run faster.

We can also run several instances of each operation. In this case, the migration will run in parallel this way speeding it up significantly. Even a single table data migration can be parallelised.


In [54]:
!curl {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "schema" }}'

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Again, waiting until this task is finished:


In [55]:
!curl -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"schema","status":"Running","messages":[{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"COUNTRIES\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"JOB_HISTORY\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating SEQUENCE \"HR\".\"EMPLOYEES_SEQ\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"JOBS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"EMPLOYEES\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"LOCATIONS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".\"REGIONS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating PROCEDURE \"HR\".\"SECURE_DML\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating SEQUENCE \"HR\".\"DEPARTMENTS_SEQ\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating SEQUENCE \"HR\".\"LOCATIONS_SEQ\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating TABLE \"HR\".

In the `init` stage the framework also queries the sizes of each table. Using them we can get the most optimal strategy for copying the data.

So let's see the sizes:


In [56]:
%%sql {PG}
select name, pg_size_pretty(data_size), stage from dbt.migration_objects where type = 'TABLE' order by data_size desc

7 rows affected.


name,pg_size_pretty,stage
EMPLOYEES,64 kB,D
LOCATIONS,64 kB,D
REGIONS,64 kB,D
JOBS,64 kB,D
JOB_HISTORY,64 kB,D
DEPARTMENTS,64 kB,D
COUNTRIES,0 bytes,D



By analysing the sizes for different tables we can apply different strategies for different tables. But size the sizes are quire small for this schema we just use the default strategy everywhere.

Now starting the data migration operation, as all the operations before, but with `data` as `stage` field value:


In [58]:
!curl {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "data" }}'

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Awaiting the operation to be completed:

In [59]:
!curl -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"data","status":"Running","messages":[{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"EMPLOYEES\"in 0.157s(exec=0.1269s,copy-stmt=0.0054s,rows=0.0001s,write=0.0053s,set-state=0.0104s)\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"JOBS\"in 0.053s(exec=0.0365s,copy-stmt=0.0004s,rows=0.0001s,write=0.0003s,set-state=0.0127s)\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"LOCATIONS\"in 0.055s(exec=0.0383s,copy-stmt=0.0005s,rows=0s,write=0.0005s,set-state=0.012s)\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"REGIONS\"in 0.08s(exec=0.0397s,copy-stmt=0.0002s,rows=0s,write=0.0001s,set-state=0.0369s)\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"JOB_HISTORY\"in 0.044s(exec=0.029s,copy-stmt=0.0003s,rows=0s,write=0.0003s,set-state=0.0118s)\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: migrating data \"HR\".\"DEPARTMENTS\"in 0.056s(exec=0.0392s,copy-stmt=0.0004s,rows=0s,wr

## Constraints
This stage adds constraints, indexes and triggers to the data we've moved in the previous step:


In [60]:
!curl {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "constraints" }}'

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

Awaiting the operation to be completed:

In [61]:
!curl -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"constrants","status":"Running","messages":[{"level":"Info","message":"^+👍^: ^gDONE!^ translating constraints for \"HR\".\"JOBS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^ translating constraints for \"HR\".\"REGIONS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^ translating constraints for \"HR\".\"COUNTRIES\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^ translating constraints for \"HR\".\"LOCATIONS\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"JHIST_JOB_IX\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"LOC_STATE_PROVINCE_IX\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"JHIST_EMPLOYEE_IX\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"LOC_CITY_IX\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"JHIST_DEPARTMENT_IX\"\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: translating INDEX \"HR\".\"EMP_MANAGER_IX\"\n"},{"level":"Info","m

Now we can verify the data in both databases are the same, by comparing each column hash sums:

In [62]:
!curl {API_PREFIX}/operation -H 'Content-Type: application/json' -d '{{ "oracle": 1, "lgw": 3, "stage": "check" }}'

{"message":"Config has been set successfully.","config":{"title":"init","status":"Running","messages":[],"progress":0}}

In [63]:
!curl -X POST {API_PREFIX}/operation/wait -H 'Content-Type: application/json'

{"title":"check","status":"Running","messages":[{"level":"Info","message":"🤞 checking \"HR\".\"EMPLOYEES\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"EMPLOYEES\"\n"},{"level":"Info","message":"🤞 checking \"HR\".\"JOBS\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"JOBS\"\n"},{"level":"Info","message":"🤞 checking \"HR\".\"REGIONS\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"REGIONS\"\n"},{"level":"Info","message":"🤞 checking \"HR\".\"LOCATIONS\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"LOCATIONS\"\n"},{"level":"Info","message":"🤞 checking \"HR\".\"JOB_HISTORY\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"JOB_HISTORY\"\n"},{"level":"Info","message":"🤞 checking \"HR\".\"DEPARTMENTS\" (0.06M)...\n"},{"level":"Info","message":"^+👍^: ^gDONE!^: checking data \"HR\".\"DEPARTMENTS\"\n"},{"level":"Info","messa

This is it, now the whole database is migrated. Please see the guide document for the information about different migration options.

## Running queries

Let's now run a few queries in the both databases to see the results are identical. There is a `checksql` stage to do this automatically, but for the demo purposes we do this manually.

So first we check the databases are indeed Oracle nad PostgreSQL

In [64]:
%%sql {ORACLE}
select banner from v$version

0 rows affected.


banner
Oracle Database 19c Enterprise Edition Release 19.0.0.0.0 - Production


In [65]:
%%sql {LGW}
select banner from v$version

1 rows affected.


BANNER
"PostgreSQL 14.9 (Debian 14.9-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit"


The output value on PostgreSQL can be configured to output exactly the same like Oracle if the application uses the data for some business logic selection.

Now let's run the same query in both databases.

In [66]:
QUERY="""
SELECT first_name  || ' ' ||  last_name full_name,
       salary + 
       NVL  (commission_pct, 0) sal_com, 
       DECODE (NVL(e.department_id, -3), 60, d.department_name, 
                                50, d.department_name, 
                                30, d.department_name, 
                                -3, 'UNKNOWN',
               'OTHER') dep,
       TO_CHAR(e.hire_date) hire_date
  FROM employees e, departments d
 WHERE d.department_id(+) = e.department_id
   AND e.last_name like 'G%'
ORDER BY first_name  ||  ' '  ||  last_name
"""

First running it on Oracle:

In [67]:
%%sql {ORACLE} 

{QUERY}


0 rows affected.


full_name,sal_com,dep,hire_date
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600,Shipping,13-JAN-08
Girard Geoni,2800,Shipping,03-FEB-08
Ki Gee,2400,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008,OTHER,17-AUG-02
Timothy Gates,2900,Shipping,11-JUL-06
William Gietz,8300,OTHER,07-JUN-02


And now running absolutely the same query but on PostgreSQL via Liberatii Gateway:

In [68]:
%%sql {LGW}

{QUERY}

8 rows affected.


FULL_NAME,SAL_COM,DEP,HIRE_DATE
Danielle Greene,9500.15,OTHER,19-MAR-07
Douglas Grant,2600.00,Shipping,13-JAN-08
Girard Geoni,2800.00,Shipping,03-FEB-08
Ki Gee,2400.00,Shipping,12-DEC-07
Kimberely Grant,7000.15,UNKNOWN,24-MAY-07
Nancy Greenberg,12008.00,OTHER,17-AUG-02
Timothy Gates,2900.00,Shipping,11-JUL-06
William Gietz,8300.00,OTHER,07-JUN-02


Here you may see the results are identical. 

The next steps are:

* Synchronise the databases using CDC so we avoid downtimes on switchover
* Replay workloads on a sandbox Oracle database and PostgreSQL via Liberatii Gateway, for testing correctness and performance


# Stage 2: Synchronise the databases

The 